In [1]:
from langchain.llms import GooglePalm
from constants import *

In [2]:
llm = GooglePalm(google_api_key= maker_suite_api,temperature = 0.7)

c:\Users\Prafful\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(llm("Imagine James Harden portraying a character in a movie. Write a synopsis for a movie where his beard is threatened and he has to eliminate the threat"))

**The Beard**

In a world where beards are everything, James Harden is a living legend. His beard is the envy of all men, and the object of desire of all women. But when a new villain emerges who threatens to destroy all beards, Harden must use all his skills and cunning to save the day.

With the help of his friends, Harden embarks on a journey to find the villain and stop him before it's too late. But the villain is powerful, and he's not going to give up his plans without a fight.

Can Harden defeat the villain and save the beards of the world? Or will his beard be the one to fall?

Find out in **The Beard**, coming to theaters this summer.


In [4]:
from langchain.utilities import SQLDatabase
db_user = 'root'
db_password = 'root'
db_host = 'localhost'
db_name = 'atliq_tshirts'

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info = 3)
print(db.get_table_names())

['discounts', 't_shirts']


c:\Users\Prafful\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\utilities\sql_database.py:278: UserWarning: This method is deprecated - please use `get_usable_table_names`.
  warnings.warn(


In [5]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [8]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm,db,verbose = True)

In [14]:
db_chain('How many Nike Shirts of size L are Available?')



> Entering new SQLDatabaseChain chain...
How many Nike Shirts of size L are Available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND size = 'L'
SQLResult: [(60,), (45,)]
Answer:60
> Finished chain.


{'query': 'How many Nike Shirts of size L are Available?', 'result': '60'}

In [16]:
# Few Shot Learning

fsl = [{
    'Question' : 'How many Nike Shirts are Available?',
    'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike'",
    'SQLResult': "Result of the SQL Query",
    'Answer':'684'
    },
    {
    'Question':'How many Nike Shirts of size L are Available?',
    'SQLQuery':"SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' and size = 'L'",
    'SQLResult':  "Result of the SQL Query",
    'Answer':'105'
    },
    {
    'Question' : 'What is the stock Quantity of NIKE size M?',
    'SQLQuery' : "SELECT SUM(stock_quantity) FROM t_shirts WHERE size = 'M' AND brand = 'Nike'",
    'SQLResult' : "Result of the SQL Query",
    'Answer' : '34'},
    {
    'Question' : 'What is the revenue generated after applying all the discounts?',
    'SQLQuery' : "select sum((price*(100-t)/100)*stock_quantity) from (select *,coalesce(lost_revenue.pct_discount,0) as t from (select price,stock_quantity,pct_discount from t_shirts left join discounts on t_shirts.t_shirt_id = discounts.t_shirt_id) as lost_revenue) as t ;",
    'SQLResult' : "Result of the SQL Query",
    'Answer' : '81861.3'
    },
    {
    'Question' : 'I Have an order for 3 M sized Nike and 12 XS sizes Van Heusen Tshirts, Can I fulfil the order?',
    'SQLQuery' : "SELECT stock_quantity FROM t_shirts WHERE size = 'M' AND brand = 'Nike' AND stock_quantity >= 3 UNION ALL SELECT stock_quantity FROM t_shirts WHERE size = 'XS' AND brand = 'Van Huesen' AND stock_quantity >= 12",
    'SQLResult' : "Result of the SQL Query",
    'Answer' : 'Yes'
    },
    {
    'Question' : 'How much money is lost due to discounts?',
    'SQLQuery' : "select sum((price*(t)/100)*stock_quantity) from (select *,coalesce(lost_revenue.pct_discount,0) as t from (select price,stock_quantity,pct_discount from t_shirts left join discounts on t_shirts.t_shirt_id = discounts.t_shirt_id) as lost_revenue) as t ",
    'SQLResult' : "Result of the SQL Query",
    'Answer' : '3115.7'
    }
    {
        "Question": "'How much revenue is generated from Nike t_shirts after applying discount?'",
        "SQLQuery" : "select sum((price*(100-pct_discount)/100)*stock_quantity) from (select price,stock_quantity,pct_discount from t_shirts left join discounts on t_shirts.t_shirt_id = discounts.t_shirt_id where brand = 'Nike') as t; ",
        'SQLResult' : "Result of the SQL Query",
        'Answer' : '2431.4'
    }
]

In [17]:
fsl_string = [ " ".join(i.values()) for i in fsl]
fsl_string

["How many Nike Shirts are Available? SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' Result of the SQL Query 684",
 "How many Nike Shirts of size L are Available? SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' and size = 'L' Result of the SQL Query 105",
 "What is the stock Quantity of NIKE size M? SELECT SUM(stock_quantity) FROM t_shirts WHERE size = 'M' AND brand = 'Nike' Result of the SQL Query 34",
 'What is the revenue generated after applying all the discounts? select sum((price*(100-t)/100)*stock_quantity) from (select *,coalesce(lost_revenue.pct_discount,0) as t from (select price,stock_quantity,pct_discount from t_shirts left join discounts on t_shirts.t_shirt_id = discounts.t_shirt_id) as lost_revenue) as t ; Result of the SQL Query 81861.3',
 "I Have an order for 3 M sized Nike and 12 XS sizes Van Heusen Tshirts, Can I fulfil the order? SELECT stock_quantity FROM t_shirts WHERE size = 'M' AND brand = 'Nike' AND stock_quantity >= 3 UNION ALL SELECT st

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

emb = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-miniLM-L6-v2')

In [19]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(fsl_string,embedding=emb, metadatas=fsl)

In [20]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(vectorstore = vectorstore, k=1)

In [21]:
example_selector.select_examples({"Question" : 'How much money is lost due to discounts?'})

[{'Answer': '3115.7',
  'Question': 'How much money is lost due to discounts?',
  'SQLQuery': 'select sum((price*(t)/100)*stock_quantity) from (select *,coalesce(lost_revenue.pct_discount,0) as t from (select price,stock_quantity,pct_discount from t_shirts left join discounts on t_shirts.t_shirt_id = discounts.t_shirt_id) as lost_revenue) as t ',
  'SQLResult': 'Result of the SQL Query'}]

In [22]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt,PROMPT_SUFFIX)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [23]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [24]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [25]:
from langchain.prompts import FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [26]:
new_chain = SQLDatabaseChain.from_llm(llm,db,verbose = True,prompt = few_shot_prompt)

In [27]:
new_chain('How many tshirts are available?')



> Entering new SQLDatabaseChain chain...
How many tshirts are available?
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts
SQLResult: [(Decimal('2891'),)]
Answer:2891
> Finished chain.


{'query': 'How many tshirts are available?', 'result': '2891'}

In [28]:
new_chain.run('How much revenue is generated from Nike Tshirts?')



> Entering new SQLDatabaseChain chain...
How much revenue is generated from Nike Tshirts?
SQLQuery:select sum(price*stock_quantity) from t_shirts where brand='Nike';
SQLResult: [(Decimal('19345'),)]
Answer:19345
> Finished chain.


'19345'

In [30]:
new_chain.run('How much revenue is generated from Nike t_shirts after applying discount?')



> Entering new SQLDatabaseChain chain...
How much revenue is generated from Nike t_shirts after applying discount?
SQLQuery:select sum((price*(100-t)/100)*stock_quantity) from (select price,stock_quantity,pct_discount from t_shirts left join discounts on t_shirts.t_shirt_id = discounts.t_shirt_id where brand = 'Nike') as t ;

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 't' in 'field list'")
[SQL: select sum((price*(100-t)/100)*stock_quantity) from (select price,stock_quantity,pct_discount from t_shirts left join discounts on t_shirts.t_shirt_id = discounts.t_shirt_id where brand = 'Nike') as t ;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [31]:
!pip install streamlit

     ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.8 MB 87.5 kB/s eta 0:00:55
     ---------------------------------------- 0.0/4.8 MB 87.5 kB/s eta 0:00:55
     ---------------------------------------- 0.0/4.8 MB 87.5 kB/s eta 0:00:55
     ---------------------------------------- 0.0/4.8 MB 85.6 kB/s eta 0:00:56
     ---------------------------------------- 0.0/4.8 MB 85.6 kB/s eta 0:00:56
     --------------------------------